# Task 1

Importing of relevant libraries

In [1]:
!pip install opencv-python
import cv2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Initialising input video
video = cv2.VideoCapture('Traffic_Laramie_1.mp4')

# Error handling
if not video.isOpened():
    print("File opening error")
    exit(0)

# Retrieving frame rate of input video
frame_rate = video.get(cv2.CAP_PROP_FPS)

# Defining codec and creating VideoWriter object with input video's frame rate
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('output_vid.avi', cv2.VideoWriter_fourcc(*'MJPG'), \
                                frame_rate, (int(video.get(3)), int(video.get(4))))

# Initialising background subtractor
backgroundSubtraction = cv2.createBackgroundSubtractorKNN(detectShadows=False)

# Initialising variables for frame differencing
isRunning = True
prev_frame = None
loopBreak = False

while isRunning:
    ret, current_frame = video.read()
    
    # Error handling
    if not ret:
        loopBreak = True
        break

    # Using only bottom half of the video
    height = current_frame.shape[0]
    bottom_half = current_frame[height//2:height, :]

    # Converting to grayscale for processing
    grayscale_frame = cv2.cvtColor(bottom_half, cv2.COLOR_BGR2GRAY)
    
    # Error handling
    if prev_frame is None:
        prev_frame = grayscale_frame
        continue

    # Computing the absolute difference between the current frame and previous frame
    frameDifference = cv2.absdiff(prev_frame, grayscale_frame)

    # Applying background subtraction to the frame difference
    foregroundMask = backgroundSubtraction.apply(frameDifference)
    
    # Making detection less sensitive
    foregroundMask = cv2.dilate(foregroundMask, None, iterations=15)
    foregroundMask = cv2.erode(foregroundMask, None, iterations=15)

    # Threshold in binary
    _, thresh_mask = cv2.threshold(foregroundMask, 0, 255, cv2.THRESH_BINARY)

    # Finding contours from the threshold
    contours, _ = cv2.findContours(thresh_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Drawing boxes for the contour
    for contour in contours:
        # Surface area limit
        if cv2.contourArea(contour) < 1500:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(current_frame, (x, y + height//2), (x+w, y+h + height//2), (0, 255, 0), 2)  

    # Saving the processed frame
    cv2.imshow('Processed Frame', current_frame)
    output_video.write(current_frame)  
    
    # Next iteration 
    prev_frame = grayscale_frame.copy()

    # Exit preview
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

video.release()
output_video.release()
cv2.destroyAllWindows()

# Task 2

In [3]:
import numpy as np

In [4]:
# Initialising input video
video = cv2.VideoCapture('Traffic_Laramie_1.mp4')

# Error handling
if not video.isOpened():
    print("File opening error")
    exit(0)

# Retrieving frame rate of input video
frame_rate = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
video_length_minutes = total_frames / frame_rate / 60

# Defining codec and creating VideoWriter object with input video's frame rate
output_video = cv2.VideoWriter('output_vid.avi', cv2.VideoWriter_fourcc(*'MJPG'), frame_rate, (int(video.get(3)), int(video.get(4))))

# Initialising background subtractor
backgroundSubtraction = cv2.createBackgroundSubtractorKNN(detectShadows=False)

object_tracker = {}
object_id = 1

# Pixels from the left edge to consider the car has crossed
cross_threshold = 50  

car_count = 0

while True:
    ret, current_frame = video.read()
    if not ret:
        break

    height, width = current_frame.shape[:2]
    bottom_half = current_frame[height//2:height, :]

    grayscale_frame = cv2.cvtColor(bottom_half, cv2.COLOR_BGR2GRAY)
    foregroundMask = backgroundSubtraction.apply(grayscale_frame)
    _, foregroundMask = cv2.threshold(foregroundMask, 250, 255, cv2.THRESH_BINARY)
    foregroundMask = cv2.dilate(foregroundMask, None, iterations=2)
    
    contours, _ = cv2.findContours(foregroundMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_centroids = []
    for contour in contours:
        if cv2.contourArea(contour) > 1500:
            x, y, w, h = cv2.boundingRect(contour)
            centroid = (int(x+w/2), int(y+h/2))
            detected_centroids.append((centroid, contour))
    
    # Update object_tracker with detected_centroids
    new_tracker = {}
    for detected in detected_centroids:
        centroid, _ = detected
        match_found = False
        for obj_id, (obj_centroid, crossed) in object_tracker.items():
            if np.linalg.norm(np.array(obj_centroid)-np.array(centroid)) < 50:
                if not crossed and centroid[0] < cross_threshold:
                    car_count += 1
                    crossed = True
                new_tracker[obj_id] = (centroid, crossed)
                match_found = True
                break
        
        if not match_found:
            new_tracker[object_id] = (centroid, False)
            object_id += 1
    
    object_tracker = new_tracker.copy()

    # Showing count on preview
    cv2.putText(current_frame, f'Total Cars: {car_count}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Processed Frame', current_frame)
    output_video.write(current_frame)

    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

video.release()
output_video.release()
cv2.destroyAllWindows()

cars_per_minute = car_count / video_length_minutes

print(f'Total number of cars: {car_count}')
print(f'Cars per minute: {cars_per_minute:.2f}')


Total number of cars: 14
Cars per minute: 4.72


In [5]:
# Initialising input video
video = cv2.VideoCapture('Traffic_Laramie_2.mp4')

# Error handling
if not video.isOpened():
    print("File opening error")
    exit(0)

# Retrieving frame rate of input video
frame_rate = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
video_length_minutes = total_frames / frame_rate / 60

# Defining codec and creating VideoWriter object with input video's frame rate
output_video = cv2.VideoWriter('output_vid.avi', cv2.VideoWriter_fourcc(*'MJPG'), frame_rate, (int(video.get(3)), int(video.get(4))))

# Initialising background subtractor
backgroundSubtraction = cv2.createBackgroundSubtractorKNN(detectShadows=False)

object_tracker = {}
object_id = 1

# Pixels from the left edge to consider the car has crossed
cross_threshold = 50  

car_count = 0

while True:
    ret, current_frame = video.read()
    if not ret:
        break

    height, width = current_frame.shape[:2]
    bottom_half = current_frame[height//2:height, :]

    grayscale_frame = cv2.cvtColor(bottom_half, cv2.COLOR_BGR2GRAY)
    foregroundMask = backgroundSubtraction.apply(grayscale_frame)
    _, foregroundMask = cv2.threshold(foregroundMask, 250, 255, cv2.THRESH_BINARY)
    foregroundMask = cv2.dilate(foregroundMask, None, iterations=2)
    
    contours, _ = cv2.findContours(foregroundMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_centroids = []
    for contour in contours:
        if cv2.contourArea(contour) > 1500:
            x, y, w, h = cv2.boundingRect(contour)
            centroid = (int(x+w/2), int(y+h/2))
            detected_centroids.append((centroid, contour))
    
    # Update object_tracker with detected_centroids
    new_tracker = {}
    for detected in detected_centroids:
        centroid, _ = detected
        match_found = False
        for obj_id, (obj_centroid, crossed) in object_tracker.items():
            if np.linalg.norm(np.array(obj_centroid)-np.array(centroid)) < 50:
                if not crossed and centroid[0] < cross_threshold:
                    car_count += 1
                    crossed = True
                new_tracker[obj_id] = (centroid, crossed)
                match_found = True
                break
        
        if not match_found:
            new_tracker[object_id] = (centroid, False)
            object_id += 1
    
    object_tracker = new_tracker.copy()

    # Showing count on preview
    cv2.putText(current_frame, f'Total Cars: {car_count}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Processed Frame', current_frame)
    output_video.write(current_frame)

    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

video.release()
output_video.release()
cv2.destroyAllWindows()

cars_per_minute = car_count / video_length_minutes

print(f'Total number of cars: {car_count}')
print(f'Cars per minute: {cars_per_minute:.2f}')


Total number of cars: 9
Cars per minute: 5.11
